# **SVR (Support Vector Regression)를 이용한 Odor 분석**

---

In [1]:
import rpy2
%load_ext rpy2.ipython

In [2]:
%%R

.libPaths("/content/drive/MyDrive/R packages")
.libPaths()

[1] "/content/drive/MyDrive/R packages" "/usr/local/lib/R/site-library"    
[3] "/usr/lib/R/site-library"           "/usr/lib/R/library"               


## **0. Packages Installation**
---

In [3]:
%%R

#install.packages("corrplot")                  
#install.packages("PerformanceAnalytics")      # 상관분석 시각화
#install.packages("caret")      
#install.packages("car")      # 다중공산성  
#install.packages("e1071")    # SVM
# install.packages("mice")    # M.I 결측치 대체
# install.packages("stringr")

library(mice)
library(stringr)
library(e1071)      
library(corrplot)
library(PerformanceAnalytics)
library(caret)
library(car)

R[write to console]: 
Attaching package: ‘mice’


R[write to console]: The following object is masked from ‘package:stats’:

    filter


R[write to console]: The following objects are masked from ‘package:base’:

    cbind, rbind


R[write to console]: corrplot 0.84 loaded

R[write to console]: Loading required package: xts

R[write to console]: Loading required package: zoo

R[write to console]: 
Attaching package: ‘zoo’


R[write to console]: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


R[write to console]: 
Attaching package: ‘PerformanceAnalytics’


R[write to console]: The following objects are masked from ‘package:e1071’:

    kurtosis, skewness


R[write to console]: The following object is masked from ‘package:graphics’:

    legend


R[write to console]: Loading required package: lattice

R[write to console]: Loading required package: ggplot2

R[write to console]: Loading required package: carData



## **1. Data Preparation** 
---

### **1) Read and Select to variable (odo)** 

In [4]:
%%R

getwd()
#od <- read.csv("drive/MyDrive/PLS_양돈 농가/s1.csv", fileEncoding = "euc-kr")
od <- read.csv("drive/MyDrive/PLS_양돈 농가/New_data.csv", fileEncoding = "euc-kr")
#od <- read.csv("drive/MyDrive/PLS_양돈 농가/s3.csv", fileEncoding = "euc-kr")
#od <- read.csv("drive/MyDrive/PLS_양돈 농가/s4.csv", fileEncoding = "euc-kr")
head(od)

  yy mm dd   ou  nh3   h2s  mm.1    dms  dmds    aca     ppa    iba    bta
1 18  4 24  300 0.56 145.0  0.09  12.80 0.025  30.60   17.57   2.78  10.80
2 18  4 24  300 0.62 310.0 19.40 462.00 0.025 916.52 1055.96 109.07 977.44
3 18  5  8  448 0.97  90.5  0.04  63.90 0.025 663.42  737.23  73.94 837.83
4 18  5  8 2080 1.02 344.0  0.04  13.00 0.630 490.79  570.61  51.09 719.27
5 18  5 15  310 1.18 312.0  0.04   0.04 0.025 404.53  468.03  43.17 573.75
6 18  5 15  669 0.98  61.4  0.04   0.04 0.025 349.63  347.86  36.76 582.25
     iva    vla    ph     pc   id    sk farm
1   3.64   7.72  2.84   0.03 0.20  3.00   YG
2 319.49 619.49 30.51 137.81 3.60 10.40   YG
3 215.64 469.79 14.61  67.52 0.94  3.92   YG
4 168.48 237.64  8.90  60.56 5.55  7.31   YG
5 157.22 205.67  9.21  69.90 0.20  5.43   YG
6 127.25 196.24  7.19  51.03 0.20  4.48   YG


In [5]:
%%R

### 측정일, 농장명 변수 제거

odo <- subset(od, select = -c(측정일, 농장명))
head(odo)
str(odo)

R[write to console]: Error in eval(substitute(select), nl, parent.frame()) : 
  object '측정일' not found

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In system("timedatectl", intern = TRUE) :
R[write to console]: 
 
R[write to console]:  running command 'timedatectl' had status 1




Error in eval(substitute(select), nl, parent.frame()) : 
  object '측정일' not found


RInterpreterError: ignored

### **2) 결측치 확인 및 처리**

#### **a. 결측값을 평균값으로 대체**

In [ ]:
# %%R


# ### 공백을 NA 값 취급
# odo <- replace(odo, odo == "", NA)


# ### 숫자변수가 문자변수로 잘못 입력된 변수를 숫자형으로 변환

# indx <- sapply(odo, is.character)
# odo[indx] <- lapply(odo[indx], function(x) as.numeric(as.character(x)))
# # str(odo)

# ### 결측치 확인

# odo<- replace(odo, odo == "ND", NA)

# if (sum(is.na(odo)) == 0) {
#     cat("결측치 개수 :", sum(is.na(odo)), "개")
# }else{
#     cat("- 평균으로 결측치 대체 - \n 결측치 대체 개수 :", sum(is.na(odo)), "개")
 
#     for(i in 1:ncol(odo)){
#         odo[is.na(odo[,i]),] <- round(mean(odo[,i], na.rm = T),3)
#         }
#     cat("\n ->", sum(is.na(odo)), "개")
# }



#### **b. ND - > 0, 결측 -> M.I**

In [ ]:
%%R

### 공백을 NA 값 취급
odo <- replace(odo, odo == "", NA)


### 결측치 확인
nd <- sum(str_count(odo, "ND"))
na <- sum(is.na(odo))

cat("NA , NA 개수 :", (na+nd), "개 \n \n")


if ((nd + na) == 0) {
    print("결측치 개수 : 0 개")
}else{
    
    # "ND" 먼저 제거 (그래야 ND와 문자열 공백(NA)구분 가능)
    cat("'ND' 개수 :", nd, "개")
    odo <- replace(odo, odo == "ND", 0)
    cat("->", sum(str_count(odo, "ND")), "개 (0으로 대체) \n \n")

    # 문자열을 숫자열로 변환(공백 문자를 NA로 바꾸기 위해서)
    indx <- sapply(odo, is.character)
    odo[indx] <- lapply(odo[indx], function(x) as.numeric(as.character(x)))

    cat("결측치 개수 :", na, "개")
    imp <- mice(odo)      # M.I 사용
    odo <- complete(imp)
    cat("\n ->", sum(is.na(odo)), "개 (M.I로 대체)")
}

### **3) Data visualizations**



In [ ]:
%%R

### 반응변수

plot(odo$complex.odor, type = "b", main = "Y")

In [ ]:
%%R

### 설명변수

par(mfrow = c(3,2))
for (i in c(1:6)) {
    plot(odo[,i], type = "b" , main = names(odo)[i+1])
}

for (i in c(7:12)) {
    plot(odo[,i], type = "b" , main = names(odo)[i+1])
}

for (i in c(12:15)) {
    plot(odo[,i], type = "b" , main = names(odo)[i+1])
}

par(mfrow = c(1,1))

In [ ]:
%%R

# as.data.frame(round(cor(odo),4))

chart.Correlation(odo, histogram = T, pch = 19)

## **2. SVR**

### **1) Model 생성**

In [ ]:
%%R

sr <- svm(complex.odor ~ ., , data = odo, cross = nrow(odo))
sr

In [ ]:
%%R

tsr <- tune.svm(complex.odor~., data = odo, tunecontrol = tune.control(cross=nrow(odo)))
tsr

### **2) $R^2$ 값 계산**

#### **(1) fitted value**

In [ ]:
%%R

y <- odo$complex.odor
fit <- tsr$best.model$fitted

cat("fitted R-square :", round(cor(y, fit)^2 , 4))

#### **(2) CV(valid) value**

In [ ]:
%%R

fitcontrol <- trainControl(method = "LOOCV")

# ?trainControl

In [ ]:
%%R
# install.packages("kernlab")
library(kernlab)
set.seed(123)

svm_model <- train(complex.odor ~ .,
                   data = odo,
                   method = "svmRadial",
                   trControl = fitcontrol)




In [ ]:
%%R

svm_model$valid